In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# define model for GPU
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [4]:
# set random seed and instantiate model
torch.manual_seed(32)
model = Model()

In [5]:
# check if model is on cuda, should not be yet
next(model.parameters()).is_cuda

False

In [6]:
# make cuda model
gpumodel = model.cuda()

In [7]:
# check if this model is on cuda, should be True
next(gpumodel.parameters()).is_cuda

True

In [8]:
# get iris dataset and perform train, test, split
df = pd.read_csv(r'C:\Users\dearm\OneDrive\Documents\PYTORCH_NOTEBOOKS\Data\iris.csv')
X = df.drop('target',axis=1).values
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

In [9]:
# convert tensors to cuda tensors
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [10]:
# create train and test loaders
trainloader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

In [11]:
# define loss and optimizer functions
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [12]:
# train the model!
import time
epochs = 100
losses = []
start = time.time()
for i in range(epochs):
    i+=1
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'TOTAL TRAINING TIME: {time.time()-start}')

epoch:  1  loss: 1.15071130
epoch: 11  loss: 0.93773133
epoch: 21  loss: 0.77982587
epoch: 31  loss: 0.60993993
epoch: 41  loss: 0.40079930
epoch: 51  loss: 0.25436318
epoch: 61  loss: 0.15053052
epoch: 71  loss: 0.10086943
epoch: 81  loss: 0.08128314
epoch: 91  loss: 0.07231426
TOTAL TRAINING TIME: 0.6518187522888184


In [13]:
# evaluate the model
correct = 0

with torch.no_grad():
    for i,data in enumerate(X_test):
        y_val = gpumodel.forward(data)
        print(f'{i+1:2}. {str(y_val):38}   {y_test[i]}')
        if y_val.argmax().item() == y_test[i]:
            correct += 1

print(f'\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct')

 1. tensor([-2.1252,  4.8064, -0.8628], device='cuda:0')   1
 2. tensor([-1.7985,  5.3098, -1.5449], device='cuda:0')   1
 3. tensor([  6.3542,   0.8438, -10.0541], device='cuda:0')   0
 4. tensor([-3.9123,  4.5958,  1.1693], device='cuda:0')   1
 5. tensor([-7.4713,  3.2021,  5.7853], device='cuda:0')   2
 6. tensor([-10.4976,   1.6459,   9.6297], device='cuda:0')   2
 7. tensor([  6.3201,   0.9917, -10.1532], device='cuda:0')   0
 8. tensor([  7.0468,   0.7059, -10.9137], device='cuda:0')   0
 9. tensor([-7.2061,  3.3477,  5.3565], device='cuda:0')   2
10. tensor([-9.3960,  2.5759,  8.1033], device='cuda:0')   2
11. tensor([-9.8808,  2.3475,  8.7141], device='cuda:0')   2
12. tensor([ 6.2748,  0.6655, -9.7613], device='cuda:0')   0
13. tensor([-9.3142,  2.1880,  8.1947], device='cuda:0')   2
14. tensor([-3.7803,  4.5050,  1.0752], device='cuda:0')   1
15. tensor([-7.8657,  3.0117,  6.2303], device='cuda:0')   2
16. tensor([-1.8867,  5.1572, -1.3345], device='cuda:0')   1
17. tensor([